<center>

# Python 中的保留字

通过下面的命令可以输出所有 python 已设定的保留字

In [1]:
import keyword

k= keyword.kwlist
print(k)

['False', 'None', 'True', '__peg_parser__', 'and', 'as', 'assert', 'async', 'await', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']


## 语法说明

- `""`中的成分是格式语句，`<>`中的成分是内容部分，例如`"import" <module>`代表代码格式应该是`import`后根具体模块名称，如`import numpy`；
- `[]`中的成分是选择性添加的部分，例如
```bash
"import" <module> ["as" <identifier>]["," <module> ["as" <identifier>]]
```
表示其形式可以是`import <module>`，也可以是`import <module> as <identifier>`，也可以是`import <module1> as <identifier1>, <module2> as <identifier2>`，如`import numpy`、`import numpy as np`、`import numpy as np, matplotlib as mpl`
- `□*`指的是□的内容可以无限叠加，如
```bash
"import" <module> ["as" <identifier>]["," <module> ["as" <identifier>]]*
```
表示后面的`["," <module> ["as" <identifier>]]`可以附加任意多个

# 

importlib

The importlib module provides a rich API for interacting with the import system. For example importlib.import_module() provides a recommended, simpler API than built-in __import__() for invoking the import machinery. Refer to the importlib library documentation for additional detail.

# 1. import 相关


## 1.1 概述

import 基本语法如下

```bash
"import" <module> ["as" <identifier>]["," <module> ["as" <identifier>]]*
"from" <relative_module> "import" <identifier> ["as" <identifier>]["," <identifier> ["as" <identifier>]]*
"from" <relative_module> "import" "(" <identifier> ["as" <identifier>]["," <identifier> ["as" <identifier>]]* [","] ")"
"from" <module> "import" "*"
```

其中`<module>`的语法格式是`[<identifier> "."]* <identifier>`；`<relative_module>`的语法格式是`"."* <module>`或`"."+`。


import 语句包含对模块的搜索、创建和名称绑定等步骤：
1. 由内置`__import__()`函数对指定模块进行搜索，并在找到时对其进行创建和初始化；期间可能会存在导入父包、更新各种缓存 (包括`sys.modules`)等操作；如果无法找到指定的模块，则会抛出`ModuleNotFoundError`异常。
2. 将搜索结果与本地作用域的名称进行绑定——只有 import 语句才执行名称绑定，直接调用`__import__()`函数只会执行搜索、创建等操作

除了 import 语句外，例如`importlib.import_module()`和内置的`__import__()`等的函数也可以调用导入机制；其中`importlib.import_module()`等函数可能会绕过`__import__()`函数，而以其自己的方式实现导入。



#### >>> **示例**

In [21]:
import numpy, math, sys, os
import math, numpy as np, sys, os as o_s
import numpy.random as random

## 1.2 包和模块

**模块 (module)** 是 Python 代码的组织单元，其包含了内部所有 Python 对象的命名空间。在 Python 中，无论一个模块是用何种语言实现的，模块对象的类型**都只有一个**。

为了能够组织模块、提供命名层次结构，Python 提出了**包 (package)** 的概念。从形式上来看，包可看作文件系统中的目录，模块可看作目录中的文件，但二者并不等价，因为包和模块不一定来自文件系统。从本质上来说，包是一种含有`__path__`属性的特殊模块；包通常是分层组织的，并含有子包和模块；模块名、子包名与父包名之间用点分隔，类似于 Python 中访问属性的语法；例如一个`email`包里的子包`email.mime`，以及该子包里的一个`email.mime.text`模块。

Python定义了两种类型的包，普通包 (regular packages) 和命名空间包 (namespace packages)


### 1.2.1 普通包

普通包中应含有`__init__.py`文件，导入这个包时会自动执行`__init__.py`文件，进而该文件所定义的对象都会绑定到包命名空间中的名称上；`__init__.py`就像 Python 其他普通模块一样，可以包含任意形式的代码，而 Python 会在导入该模块时对其添加一些额外属性。例如下面的文件结构表示一个位于顶层的父包和三个子包：

```bash
parent/
    __init__.py
    one/
        __init__.py
    two/
        __init__.py
    three/
        __init__.py
```


### 1.2.2 命名空间包

名称空间包有各种 portion 构成，其中每个 portion 可以为父包提供一个子包；portion 可以位于zip 文件、网上、文件系统的其他位置、或其他任何 Python 会进行搜索的位置。命名空间包可能直接对应文件系统上的对象，也可以是没有具体表示的虚拟模块。

命名空间包的`__path__`属性不是普通的列表类型，而是一个自定义的可迭代类型，该迭代类型会在父包(或顶级包的`sys.path`)的路径改变后的尝试导入中对 portion 进行新的搜索。

命名空间包也没有`parent/__init__.py`文件。由于在导入的搜索期间可能会发现多个`parent`目录，其中每个目录由不同的 portion 提供，因此`parent/one`未必与`parent/two`相邻，此时无论导入顶层的`parent`包还是其子包，Python 都会为`parent`包创建一个命名空间包。

更多有关命名空间包的说明可参见 [PEP 420](https://www.python.org/dev/peps/pep-0420)

## 1.3 搜索

Python 需要获得导入模块的完全限定名来对其进行搜索，该名称会用于搜索的各个阶段；这里完全限定名是一个点名称，表示一个从模块的全局作用域到该模块中定义的子模块、类、函数或方法的“路径”，例如`foo.bar.baz`，此时 Python 会先后尝试导入`foo`、`foo.Bar`、`foo.bar.baz`，如果中间任一导入失败，都会抛出`ModuleNotFoundError`；


### 1.3.1 `sys.modules`缓存

搜索过程中检查的第一个位置便是字典对象`sys.modules`，该映射含有之前导入的所有模块的缓存及其中间路径，例如之前导入了`foo.bar.baz`，则`sys.modules`会包含`foo`、`foo.bar`、`foo.bar.baz`，其中每个键都有模块对象作为其相应的值。

在导入期间，Python 会先在`sys.modules`中查询相关模块名，名称存在时则导入对应值，如果该对应值为`None`，则会抛出`ModuleNotFoundError`异常；而如果没有该模块名，Python 会继续在其他位置进行搜索

`sys.modules`是可写入的，删除某个键未必会删除相关模块，因为该模块可能被其他模块引用；但由于其对应的缓存条目失效，进而 Python 在下次导入时需要重新搜索；需要说明的是，如果此时对该模块的引用还在，重新导入的模块与原模块并非*同一对象*；若想使用同一对象，可以调用`importlib.reload


### 1.3.2 
……（https://docs.python.org/3.7/reference/import.html#fnmo）

- 最基本的`import`语句的执行包括两个步骤：

    1. 找到该模块并对其进行加载，以及在必要时进行初始化，相关具体内容参见 [import system](https://docs.python.org/3.7/reference/import.html#importsystem)；此步骤报错的原因可能是指定导入的模块无法定位，或是初始化模块时发生了错误，这里初始化包括了对模块代码的执行；
    2. 在本地命名空间中为`import`所在的作用域定义一个或多个名称，其方式为下述之一：
        1. 若模块名后面跟着`as`，那么`as`后面的名称将与导入模块绑定；
        2. 如果没有指定其他名称，并且被导入的模块位于 top level，则该名称会绑定在本地命名空间中，并作为对被导入模块的引用
        3. 如果被导入的模块不位于 top level，则包含该模块且位于 top level 的包名称会绑定在本地命名空间中，作为对顶级包的引用，并只能通过其完整限定名调用导入的模块，而不能直接使用其名称

    当`import`语句包含多由逗号隔离的子句时，会分别对每个子句执行上述 2 个步骤。

- `from`形式语句的执行过程会略加复杂：

    1. 找到`from`语句里指定的模块并对其进行加载，以及在必要时进行初始化
    2. 对每个`import`语句里指定标识符：
        1. 检查导入的模块是否有该名称的属性
        2. 如果没有，尝试导入具有该名称的子模块，然后再次检查导入的模块是否有该属性
        3. 如果没有找到该属性，抛出`ImportError`
        4. 如果找到了该属性，则本地命名空间中储存一个对该变量的引用；如果有`as`则使用`as`指定的名称，否则使用属性的名称


```python
import foo                 # foo imported and bound locally
import foo.bar.baz         # foo.bar.baz imported, foo bound locally
import foo.bar.baz as fbb  # foo.bar.baz imported and bound as fbb
from foo.bar import baz    # foo.bar.baz imported and bound as baz
from foo import attr       # foo imported and foo.attr bound as attr
```





对于`from <module> import *`的情况，该模块中定义的所有公共名称都被绑定到`import`所在作用域的本地命名空间中；其中“公共名称”是通过模块的命名空间中一个名为`__all__`的变量定义的，其类型应是字符串列表，每个字符串代表随该模块导入的名称，且`__all__`中的名称都应当是公共的；如果人为没有定义`__all__`，公共名称默认为该模块的命名空间中所有不以下划线`_`开头的名称；`__all__`应该包含所有公共 API，以避免导出非 API 项 (例如在该模块中导入和使用的其他库模块)。这种通配符形式的导入语句仅限于在模块层级使用，在类或函数中使用这类语句会抛出`SyntaxError`。


当一个模块或包存在于另一个包时，可以在相同的顶层包中进行相对导入；通过在`from`后面使用前导点`.`，可以指定向上遍历当前包层次结构的高度；例如在`pkg`包里执行`from . import mod`相当于导入`pkg.mod`；在`pkg.subpkg1`中执行`from ..subpkg2 import mod`，则会导入`pkg.subpkg2.mod`。相对导入的具体说明可参见[Package Relative Imports](https://docs.python.org/3.7/reference/import.html#relativeimports)

`importlib.import_module()`用于帮助程序动态地确定要加载的模块。


## 1.1 Future statements


future 语句可以告诉编译器是否使用 Python 未来特定版本中的语法或语义来编译，进而可以在标准版本发布新特性之前，提前对其进行使用。除注释、空行、其他 future 语句外，future 语句前不能出现其他代码；其语法如下：


```python
"from" "__future__" "import" feature ["as" identifier]("," feature ["as" identifier])*
"from" "__future__" "import" "(" feature ["as" identifier]("," feature ["as" identifier])* [","] ")"
```

由 future 语句启用的历史特性仍然可以被 Python 3 识别，其中包括`absolute_import`、`division`、`generators`、`generator_stop`、`unicode_literals`、`print_function`、`nested_scopes`、`with_statement`；实际上无需对上述模块进行导入，因为总是它们的功能总是处于启用状态，并且只是为了向后兼容才对其进行保留。

future 语句在编译时会以一种特殊的方式进行：对核心结构的语义的修改会通过不同代码来实现；如果新特性引入了不兼容语法，如新的保留字等，编译器会在 runtime 之前决定出以哪种不同的方式对模块进行解析；对于给定的版本，编译器知道那些已经定义了的特性名称，如果 future 语句中包含了其不认识的特性，就会抛出 compile-time 错误。


需要注意的是，`import __future__ [as name]`并非 future 语句，而仅仅是普通的 import 语句，没有特殊的语义或语法限制


如果调用含有 future 语句的模块 M 中的内置函数`exec()`和`compile()`所编译的代码，会默认使用与 future 语句相关的新语法或语义，这可以通过`compile()`参数来调节，详细信息请参阅该函数的文档。


在交互式解释器中输入 future 语句只会对随后的会话生效；如果解释器通过`-i`选项启动，并传递了一个含有 future 语句的脚本名称，则 future 语句会在脚本执行后的会话中生效。


更多内容可参见 **`PEP 236`** —— 提出`__future__`机制的原始文档

# 

In [1]:
import keyword

k= keyword.kwlist
print(k)

['False', 'None', 'True', 'and', 'as', 'assert', 'async', 'await', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']


# The "with" statement

### 相关概念

- **上下文管理协议** Context Management Protocol：包含方法`enter()`和`exit()`，支持该协议的对象至少要实现这两个方法

- **上下文表达式** Context Expression：with 语句中跟在关键字`with`之后的表达式，该表达式返回一个**上下文管理器**对象

- **上下文管理器** Context Manager：支持上下文管理协议的对象，由上下文表达式返回；如上所述，这种对象实现了`enter()`和 `exit()` 方法；**上下文管理器**在定义执行 with 的语句时，需要建立**运行时上下文**，其负责执行 with 语句块上下文中的进入与退出操作。通常使用 with 语句来调用上下文管理器，也可以通过直接调用等其方法来使用

- **运行时上下文** runtime context：由上下文管理器创建，通过上下文管理器的`enter()`和 `exit()`方法实现，`enter()`方法在**语句体**执行之前进入**运行时上下文**，`exit()`在**语句体**执行完后从**运行时上下文**退出

- **语句体** with-body：with 语句包裹起来的代码块

### with 语法及原理

with 语句用于对代码块的执行进行包装，这些代码块通常含有上下文管理器定义的函数；代码块允许对通常的`try - except - finally`语句使用的封装，以便重新调用。语法格式如下：

```python
with <context_expression>[ as <target(s)>, context_expression2 [as <target(s)2>]]:
    suite(i.e. with_body)
```
######  

以下两种语句是等价的：

```python
with A() as a, B() as b:
    suite

with A() as a:
    with B() as b:
        suite
```
######   

带有一个 context_expression 的 with 语句的执行过程如下：

1. 上下文表达式 Context Expression 被识别、计算，得到一个上下文管理器 Context Manager

2. 加载上下文管理器的 exit() 方法以备调用

3. 调用上下文管理器的 enter() 方法: ``

4. 若赋值目标被包括在 with 语句中，即`as <target(s)>`语句，则将 enter() 的返回值赋给它，`target(s)`可以是单个变量，或者由”()”括起来的元组，随后执行语句体: 

5. 调用上下文管理器的 exit() 方法

内部运行代码大致如下所示：

```python
context_manager = context_expression
exit = type(context_manager).__exit__
value = type(context_manager).__enter__(context_manager)
exc = True  # if to excute
try:
    try:
        target = value  # only if "as" statement is present
        with_body
    except:
        # The exceptional case is handled here
        exc = False
        if not exit(context_manager, *sys.exc_info()):
            raise
        # The exception is swallowed if exit()
return True

finally:
    # The normal and non-local-goto cases are handled here
    if exc:
        exit(context_manager,
             exc_type=None,
             exc_value=None,
             exc_traceback=None)
```
由上可以看出，由于当`enter()`方法抛出异常时，with 语句能够确保随即调用`exit()`函数，因此如果在给目标列表赋值时抛出异常，该异常将被等同地视为在语句体中发生的异常；

若有异常使得语句体退出，则该异常对象的类型、取值、回溯(traceback)将作为参数传递给`exit()`函数，否则将传递 3 个 None 值；
    
若有异常使得语句体退出，且`exit()`函数返回值为`False`时，异常被抛出，进而转接给 with 语句块之外的语句处理；若返回值为`True`时，异常将被抑制，并继续执行“with”语句块后面的语句
    
若语句体因为不同于异常的其他原因退出，`exit()`函数返回值将被忽略，且程序在退出的位置正常执行(execution proceeds at the normal location for the kind of exit that was taken)


**See also**

[PEP 343 -- The “with” statement](https://www.python.org/dev/peps/pep-0343/)

**Examples**

Python 对一些 builtin 对象进行改进，加入了对上下文管理器的支持，可以用于 with 语句中，比如利用 with 语句可以实现自动关闭文件、线程锁的自动获取和释放等；假设要对一个文件进行操作，使用 with 语句可以有如下代码：

In [ ]:
with open(r'./test_files/with_open_test.txt') as f:
    for line in f:
        print(line)

这里使用了 with 语句，不管在处理文件过程中是否发生异常，都能保证 with 语句执行完毕后执行关闭打开文件的句柄；

如果使用传统的 try-finally 范式，则要使用类似如下代码：

In [ ]:
f = open(r'./test_files/with_open_test.txt')
try:
    for line in f:
        print(line)
finally:
    f.close()

下面通过一个简单的示例来演示如何构建自定义的上下文管理器。注意，上下文管理器必须同时提供 enter() 和 exit() 方法的定义，缺少任何一个都会导致`AttributeError`；with 语句运行时也会先检查是否提供了`exit()`方法，然后检查是否定义了`enter()`方法。

假设有一个资源 DummyResource，这种资源需要在访问前先分配，使用完后再释放掉；分配操作可以放到`enter()`方法中，释放操作可以放到`exit()`方法中；简单起见，这里只通过打印语句来表明当前的操作，并没有实际的资源分配与释放。

In [ ]:
class DummyResource():
    def __init__(self, tag):
        self.tag = tag
        print("the __init__ function called\nResource [{}]\n".format(tag))
    def __enter__(self):
        print("the __enter__ function called\n[Enter {}]: Allocate resource\n".format(self.tag))
        return self  # can be any different type as needed, not necessarily the context manager object itself
    def __exit__(self, exc_type=None, exc_value=None, exc_tb=None):
        print("the __exit__ function called\n[Exit {}]: Free resource".format(self.tag))
        if exc_tb is None:
            print("[Exit {}]: Exited without exception".format(self.tag))
        else:
            print("[Exit {}]: Exited with exception raised".format(self.tag))
            return False  # can be omitted, since the default `None` is also considered as `False`
with DummyResource("Normal"):
    print("running the [with_body] without exceptions.\n")

In [ ]:
with DummyResource("Exception"):
    print("running the [with_body] with an exception coming.\n")
    raise Exception
    print("and an exception was raised. Failed to finish statement body")

 with 语句的优势在于
- 省略简化代码
- 解决异常退出时资源释放的问题
- 解决用户忘记调用close方法而产生的资源泄漏问题
- 能够对一些的异常进行处理，保证代码顺利进行

可见所谓上下文管理器、上下文管理协议、运行时上下文等等，目的是将一些代码块嵌套在整个程序中，在简化代码、自动处理某些异常的同时，还能够避免一些错误发生；

#  

#  

# The "assert" statement

断言语句 (assert statements) 是将调试断言 (debugging assertions) 插入程序的一种方便方法

   assert_stmt ::= "assert" expression ["," expression]

The simple form, "assert expression", is equivalent to

   if __debug__:
       if not expression: raise AssertionError

The extended form, "assert expression1, expression2", is equivalent to

   if __debug__:
       if not expression1: raise AssertionError(expression2)

These equivalences assume that "__debug__" and "AssertionError" refer
to the built-in variables with those names.  In the current
implementation, the built-in variable "__debug__" is "True" under
normal circumstances, "False" when optimization is requested (command
line option "-O").  The current code generator emits no code for an
assert statement when optimization is requested at compile time.  Note
that it is unnecessary to include the source code for the expression
that failed in the error message; it will be displayed as part of the
stack trace.

Assignments to "__debug__" are illegal.  The value for the built-in
variable is determined when the interpreter starts.

In [ ]:
x = [[2, 3], [4, 5]]
assert type(x) == list
assert type(x[0][0]) == int
# assert type(x) == bool  # 运行会报错

In [ ]:
help("lambda")

#  

#  

# The "try" statement
*******************

“try” 语句为一组语句的运行指定了异常处理程序和/或善后代码(cleanup code)
```
try_stmt = try1_stmt | try2_stmt
try1_stmt :
try: suite
(except[expression ["as" identifier]]: suite)
(else: suite)
(finally: suite)

try2_stmt = try: suite
finally: suite
```

“except”语句指定一个或多个异常处理程序。如果“try”语句中没有发生异常，则不执行异常处理程序；当“try”语句中出现异常时，则将开始依次检查except语句，直到找到与异常匹配的语句为止。

对于没有“except”语句，必须在所有“try”语句最后，其可以匹配任何异常；对于带有“except”表达的语句，检测到异常时将运行表达，如果运行结果对象与指定异常兼容，则子句将匹配异常，这里匹配是指，该对象是异常对象的类或基类，或包含与异常兼容的项的元组

If no except clause matches the exception, the search for an exception handler continues in the surrounding code and on the invocation stack.

If the evaluation of an expression in the header of an except clause raises an exception, the original search for a handler is canceled and a search starts for the new exception in the surrounding code and on the call stack (it is treated as if the entire "try" statement raised the exception).

When a matching except clause is found, the exception is assigned to the target specified after the "as" keyword in that except clause, if present, and the except clause’s suite is executed.  All except clauses must have an executable block.  When the end of this block is reached, execution continues normally after the entire try statement. (This means that if two nested handlers exist for the same exception, and the exception occurs in the try clause of the inner handler, the outer handler will not handle the exception.)

When an exception has been assigned using "as target", it is cleared
at the end of the except clause.  This is as if

   except E as N:
       foo

was translated to

   except E as N:
       try:
           foo
       finally:
           del N

This means the exception must be assigned to a different name to be able to refer to it after the except clause.  Exceptions are cleared because with the traceback attached to them, they form a reference cycle with the stack frame, keeping all locals in that frame alive until the next garbage collection occurs.

Before an except clause’s suite is executed, details about the exception are stored in the "sys" module and can be accessed via "sys.exc_info()". "sys.exc_info()" returns a 3-tuple consisting of the exception class, the exception instance and a traceback object (see section The standard type hierarchy) identifying the point in the program where the exception occurred.  "sys.exc_info()" values are restored to their previous values (before the call) when returning from a function that handled an exception.

The optional "else" clause is executed if the control flow leaves the "try" suite, no exception was raised, and no "return", "continue", or "break" statement was executed.  Exceptions in the "else" clause are not handled by the preceding "except" clauses.

If "finally" is present, it specifies a ‘cleanup’ handler.  The "try" clause is executed, including any "except" and "else" clauses.  If an exception occurs in any of the clauses and is not handled, the exception is temporarily saved. The "finally" clause is executed.  If there is a saved exception it is re-raised at the end of the "finally" clause.  If the "finally" clause raises another exception, the saved exception is set as the context of the new exception. If the "finally" clause executes a "return" or "break" statement, the saved exception is discarded:

   >>> def f():
   ...     try:
   ...         1/0
   ...     finally:
   ...         return 42
   ...
   >>> f()
   42

在执行“finally”语句期间，程序无法获得异常信息

当“try...finally”语句的“try”语句中执行了“return”、“break”或“continue”语句时，“finally”子句也会在退出时执行，“continue”语句不能在“finally”语句中出现(原因是目前程序执行存在问题，这一限制可能在将被取消)；函数的返回值由最后执行的“return”语句决定，即由于“finally”子句总是执行的，所以会“finally”子句中“return”语句的返回值


   >>> def foo():
   ...     try:
   ...         return 'try'
   ...     finally:
   ...         return 'finally'
   ...
   >>> foo()
   'finally'

关于异常的其他信息可以在`Exceptions`章节找到，关于使用“raise”语句生成异常的信息可以在`The raise statement`章节找到。

Related help topics: EXCEPTIONS

#  

#  

# The "del" statement

`del target_list`

删除语句与赋值语句方式非常相似，均是递归定义的；这里仅提供一些提示而非详细的说明。

递归地删除目标列表会从左到右删除每个目标

删除名称将从本地或全局名称空间中删除该名称的绑定，具体取决于该名称是否出现在同一代码块中的global语句中；如果未绑定名称，将抛出“NameError”异常。

属性引用、subscriptions和切片的删除会被传递给所涉及的主对象，删除切片通常等价于给原对象分配一个正确类型的空切片

#  

#  

# Function definitions

函数定义可以定义用户自定义的函数对象(参见 The standard type hierarchy 一节)

```python
[decorators] def <func_name>([param_list])
    [-> expression]: suite
```
其中 decorator 形式为`@ dotted_name [( [argument_list [,] )] NEWLINE`，`dotted_name`形式为`identifier(.identifier)*`； 

   funcdef                 ::= [decorators] "def" funcname "(" [parameter_list] ")"
               ["->" expression] ":" suite
   decorators              ::= decorator+
   decorator               ::= "@" dotted_name ["(" [argument_list [","]] ")"] NEWLINE
   dotted_name             ::= identifier ("." identifier)*
   parameter_list          ::= defparameter ("," defparameter)* ["," [parameter_list_starargs]]
                      | parameter_list_starargs
   parameter_list_starargs ::= "*" [parameter] ("," defparameter)* ["," ["**" parameter [","]]]
                               | "**" parameter [","]
   parameter               ::= identifier [":" expression]
   defparameter            ::= parameter ["=" expression]
   funcname                ::= identifier

一个函数定义是一个可执行语句；执行该语句会将当前 local 命名空间中的函数名绑定到函数对象上，即一个可供函数执行代码的包装器；此函数对象包含对当前 global 命名空间的引用，这是因为调用函数时要用到 global 命名空间

一个函数的定义可以由一个或多个 decorator 表达式包装；decorator 表达式会在包含了函数定义的作用域中，在定义函数时被计算；计算结果必须是可调用的函数名称， 
。结果必须是可调用的，它是用函数对象作为唯一参数调用的。返回值被绑定到函数名，而不是函数对象。多个decorator以嵌套方式应用。例如下面两种代码，除原函数并不是暂时的绑定在`func`名称上之外，二者大致是等价的
```python
@f1(arg)
@f2
def func(): pass


def func(): pass
func = f1(arg)(f2(func))
```




A function definition may be wrapped by one or more decorator expressions. Decorator expressions are evaluated when the function is defined, in the scope that contains the function definition.  The result must be a callable, which is invoked with the function object as the only argument. The returned value is bound to the function name instead of the function object.  Multiple decorators are applied in nested fashion. For example, the following code is roughly equivalent except that the original function is not temporarily bound to the name "func".

当一个或多个参数具有形式`parameter=expression`时，代表此参数具有默认值，进而在调用函数时该参数的传参可以省略，进而使用该默认值；如果一个参数有一个默认值，那么在其之后到`*`之前所有参数也必须设定一个默认值

**Default parameter values are evaluated from left to right when the
function definition is executed.** This means that the expression is
evaluated once, when the function is defined, and that the same “pre-
computed” value is used for each call.  This is especially important
to understand when a default parameter is a mutable object, such as a
list or a dictionary: if the function modifies the object (e.g. by
appending an item to a list), the default value is in effect modified.
This is generally not what was intended.  A way around this is to use
"None" as the default, and explicitly test for it in the body of the
function, e.g.:

   def whats_on_the_telly(penguin=None):
       if penguin is None:
           penguin = []
       penguin.append("property of the zoo")
       return penguin

Function call semantics are described in more detail in section Calls.
A function call always assigns values to all parameters mentioned in
the parameter list, either from position arguments, from keyword
arguments, or from default values.  If the form “"*identifier"” is
present, it is initialized to a tuple receiving any excess positional
parameters, defaulting to the empty tuple. If the form
“"**identifier"” is present, it is initialized to a new ordered
mapping receiving any excess keyword arguments, defaulting to a new
empty mapping of the same type.  Parameters after “"*"” or
“"*identifier"” are keyword-only parameters and may only be passed
used keyword arguments.

Parameters may have an *annotation* of the form “": expression"”
following the parameter name.  Any parameter may have an annotation,
even those of the form "*identifier" or "**identifier".  Functions may
have “return” annotation of the form “"-> expression"” after the
parameter list.  These annotations can be any valid Python expression.
The presence of annotations does not change the semantics of a
function.  The annotation values are available as values of a
dictionary keyed by the parameters’ names in the "__annotations__"
attribute of the function object.  If the "annotations" import from
"__future__" is used, annotations are preserved as strings at runtime
which enables postponed evaluation.  Otherwise, they are evaluated
when the function definition is executed.  In this case annotations
may be evaluated in a different order than they appear in the source
code.

It is also possible to create anonymous functions (functions not bound
to a name), for immediate use in expressions.  This uses lambda
expressions, described in section Lambdas.  Note that the lambda
expression is merely a shorthand for a simplified function definition;
a function defined in a “"def"” statement can be passed around or
assigned to another name just like a function defined by a lambda
expression.  The “"def"” form is actually more powerful since it
allows the execution of multiple statements and annotations.

**Programmer’s note:** Functions are first-class objects.  A “"def"”
statement executed inside a function definition defines a local
function that can be returned or passed around.  Free variables used
in the nested function can access the local variables of the function
containing the def.  See section Naming and binding for details.

See also:

  **PEP 3107** - Function Annotations
     The original specification for function annotations.

  **PEP 484** - Type Hints
     Definition of a standard meaning for annotations: type hints.

  **PEP 526** - Syntax for Variable Annotations
     Ability to type hint variable declarations, including class
     variables and instance variables

  **PEP 563** - Postponed Evaluation of Annotations
     Support for forward references within annotations by preserving
     annotations in a string form at runtime instead of eager
     evaluation.

Related help topics: class

In [5]:
help("@")

Function definitions
********************

A function definition defines a user-defined function object (see
section The standard type hierarchy):

   funcdef                 ::= [decorators] "def" funcname "(" [parameter_list] ")"
               ["->" expression] ":" suite
   decorators              ::= decorator+
   decorator               ::= "@" dotted_name ["(" [argument_list [","]] ")"] NEWLINE
   dotted_name             ::= identifier ("." identifier)*
   parameter_list          ::= defparameter ("," defparameter)* ["," [parameter_list_starargs]]
                      | parameter_list_starargs
   parameter_list_starargs ::= "*" [parameter] ("," defparameter)* ["," ["**" parameter [","]]]
                               | "**" parameter [","]
   parameter               ::= identifier [":" expression]
   defparameter            ::= parameter ["=" expression]
   funcname                ::= identifier

A function definition is an executable statement.  Its execution binds
the functio